In [ ]:
%matplotlib inline

import os 
import numpy as np
import time
import imp
import cv2

from datasets.motdb import *
from datasets.videodb import rank_nearest_boxes, merge_roidbs
from track_rcnn.minibatch import get_minibatch

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from JSAnimation.IPython_display import display_animation

from utils.visualize import *

# Utilities

In [ ]:
# Dataset configuration
data_path = 'data/2DMOT2015/'
train_videos = ['ADL-Rundle-6',
                'ADL-Rundle-8',
                'ETH-Bahnhof',
                'ETH-Pedcross2',
                'ETH-Sunnyday',
                'KITTI-13',
                'KITTI-17',
                'PETS09-S2L1',
                'TUD-Campus',
                'TUD-Stadtmitte', 
                'Venice-2']
test_videos = ['ADL-Rundle-1',
               'ADL-Rundle-3',
               'AVG-TownCentre',
               'ETH-Crossing',
               'ETH-Jelmoli',
               'ETH-Linthescher',
               'KITTI-16',
               'KITTI-19',
               'PETS09-S2L2',
               'TUD-Crossing',  
               'Venice-1']
videos = ['PETS09-S2L1']

# Visualize MotDB

In [ ]:
# Configuration
path_config = os.path.join('track_rcnn', 'config.py') 
config = imp.load_source('*', path_config) 
cfg = config.cfg

# Load the MOT dataset
videodb = MotDB(videos)
videodb.load_trajdb()
gt_roidb = videodb.gt_roidb()
ss_roidb = videodb.selective_search_roidb()

roidb = merge_roidbs(gt_roidb, ss_roidb)

In [ ]:
def vis_traj(traj, roidb, time_span=None, n_rois=10, plot_th=0.0, show_score=False):
    """
    Viasualize a trajectory. 
    """
    if time_span is None:
        time_span = (0, len(traj))
    if len(time_span) == 2:
        time_span = (time_span[0], time_span[1], 1)
    
    obj_id = traj[0]['obj_id']
        
    for t in range(time_span[0], time_span[1], time_span[2]):
        assert t < len(traj), 'Error: Time step %d exceeds trajectory length %d' % (t, len(traj))
        traj_t = traj[t]
        # Create figure
        figsize = (10, 10)
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(1, 1, 1)
        # Show the image
        img = cv2.imread(traj_t['image'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ax.imshow(img)
        plt.title('Image: {}\nFrame: {}, Obj_ID: {}, BBox: {}, Visible: {}'.\
            format(traj_t['image'], traj_t['fr']+1, traj_t['obj_id']+1, traj_t['gt_box'].tolist(), traj_t['visible']))
        
        # Plot the ground truth bounding box
        # noise = np.array([300, 300, 240, 270])
        noise = np.array([0, 0, 0, 0])
        plot_box(ax, traj_t['gt_box'] + noise, 1, traj_t['visible'], show_score)
        
        # Plot nearest rois
        boxes = roidb[t]['boxes']
        i_obj_id = roidb[t]['obj_ids'].index(obj_id)
        scores = roidb[t]['gt_overlaps'][:, i_obj_id]
        
        if n_rois is None:
            n_rois = traj_t['gt_box'].shape[0]
        rank_inds = rank_nearest_boxes(traj_t['gt_box'] + noise, boxes, np.array([[1, 1, 3, 3]]))
        
        for i in range(n_rois):
            ind = rank_inds[i]
            if scores[ind] >= plot_th:
                plot_box(ax, boxes[ind], scores[ind], True, show_score)
        

In [ ]:
print(len(videodb.trajdb))
print([(i, len(trajdb)) for i, trajdb in enumerate(videodb.trajdb)])

In [ ]:
ind = 4

trajdb = videodb.trajdb[ind]
print(len(trajdb))
trajdb = trajdb[:2]
video_id = trajdb[0]['video_id']
fr = trajdb[0]['fr']
roidb = ss_roidb[video_id][fr:]
vis_traj(trajdb, roidb, None, 512, False)

# Visualize Minibatch

In [ ]:
idx = 200
trajdb = motdb.trajdb[idx]
batch_data = get_minibatch(cfg, trajdb[0:10], roidb, random=True)
n_steps = batch_data['im_inputs'].shape[0]

for t in range(2):
    ims = batch_data['im_inputs'][t, :]
    rois = batch_data['roi_inputs'][t, :]
    scores = batch_data['score_targets'][t, :]
    vis_frame(cfg, ims, rois, scores, show_score=False)